In [ ]:
import time
import warnings

import pandas as pd
from memory_profiler import memory_usage
from sklearn.base import clone
from sklearn.datasets import make_classification

# --- Your other imports ---
from skrebate import SURF, ReliefF
from skrebate import MultiSURF as SkrebateMultiSURF

from src.fast_select.MultiSURF import MultiSURF as FastMultiSURF
from src.fast_select.ReliefF import ReliefF as FastReliefF
from src.fast_select.SURF import SURF as FastSURF

# --- GPU Detection ---
try:
    from numba import cuda
    GPU_AVAILABLE = cuda.is_available()
except (ImportError, cuda.cudadrv.error.CudaSupportError):
    GPU_AVAILABLE = False

# --- Benchmark Configuration ---
P_DOMINANT_SCENARIOS = {
    "name": "p >> n (Features Dominant)",
    "fixed_param": "n_samples", "fixed_value": 500,
    "varied_param": "n_features", "varied_range": [100, 200, 300, 400, 500]
}
N_DOMINANT_SCENARIOS = {
    "name": "n >> p (Samples Dominant)",
    "fixed_param": "n_features", "fixed_value": 100,
    "varied_param": "n_samples", "varied_range": [500, 1000, 1500, 2000, 2500]
}
N_FEATURES_TO_SELECT = 10
N_REPEATS = 3 # Increase repeats for more stable results

# --- Estimators to Test ---
estimators = {
    # skrebate estimators
    "skrebate.ReliefF": ReliefF(n_features_to_select=N_FEATURES_TO_SELECT,
                                n_neighbors=10, n_jobs=-1),
    "skrebate.SURF": SURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    "skrebate.MultiSURF": SkrebateMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT,
                                            n_jobs=-1),
    # fast-select CPU estimators
    "fast_select.ReliefF (CPU)": FastReliefF(n_features_to_select=N_FEATURES_TO_SELECT,
                                             n_neighbors=10, backend='cpu', n_jobs=-1),
    "fast_select.SURF (CPU)": FastSURF(n_features_to_select=N_FEATURES_TO_SELECT,
                                       n_jobs=-1),
    "fast_select.MultiSURF (CPU)": FastMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT,
                                                 backend='cpu', n_jobs=-1),
}
if GPU_AVAILABLE:
    print("NVIDIA GPU detected. Including GPU benchmarks.")
    estimators.update({
        "fast_select.ReliefF (GPU)": FastReliefF(n_features_to_select=N_FEATURES_TO_SELECT,
                                                 n_neighbors=10, backend='gpu'),
        "fast_select.SURF (GPU)": FastSURF(n_features_to_select=N_FEATURES_TO_SELECT,
                                           backend='gpu'),
        "fast_select.MultiSURF (GPU)": FastMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT,
                                                     backend='gpu'),
    })
else:
    print("No NVIDIA GPU detected. Skipping GPU benchmarks.")

# --- CORRECTED BENCHMARK FUNCTION ---
def run_single_benchmark(estimator, X, y):
    """
    Measures runtime and peak memory usage of a single estimator fit.
    This version performs only ONE execution and correctly measures GPU memory.
    """
    is_gpu_estimator = hasattr(estimator, 'backend') and estimator.backend == 'gpu'

    # Use a lambda to wrap the fit call
    fit_func = lambda: estimator.fit(X, y)

    # --- Memory Measurement ---
    peak_mem_mb = 0
    if is_gpu_estimator:
        # For GPU, we measure VRAM usage directly.
        # This requires the fit function to be run inside the context.
        ctx = cuda.current_context()
        start_mem = ctx.get_memory_info().free
        fit_func() # Run the function
        end_mem = ctx.get_memory_info().free
        # Peak memory is the reduction in free memory.
        peak_mem_mb = (start_mem - end_mem) / (1024**2)
    else:
        # For CPU, memory_profiler works perfectly.
        mem_samples = memory_usage(fit_func, interval=0.1)
        peak_mem_mb = max(mem_samples)

    # --- Runtime Measurement ---
    # Since the function has already been run once for memory profiling,
    # we time a second run to get a pure execution time without JIT overhead.
    # This is now a consistent measurement.
    start_time = time.perf_counter()
    fit_func()
    end_time = time.perf_counter()
    runtime = end_time - start_time

    return runtime, peak_mem_mb

def warmup_jit_compilers(estimators_dict):
    """Performs a 'warm-up' run on a small dataset to compile JIT functions."""
    print("\n--- Warming up JIT compilers ---")
    X_warmup, y_warmup = make_classification(n_samples=20, n_features=20, random_state=42)
    for name, estimator in estimators_dict.items():
        # More robust check for our custom estimators
        if "fast_select" in name:
            print(f"  Warming up {name}...")
            try:
                # Use a fresh clone for warmup
                clone(estimator).fit(X_warmup, y_warmup)
            except Exception as e:
                warnings.warn(f"  > Warm-up FAILED for {name}. Reason: {type(e).__name__}: {e}")
    print("--- Warm-up complete ---")

def main():
    """Main function to run all benchmark scenarios."""
    results = []
    warmup_jit_compilers(estimators)

    scenarios = [P_DOMINANT_SCENARIOS, N_DOMINANT_SCENARIOS]

    for scenario_params in scenarios:
        scenario_name = scenario_params["name"]
        print(f"\n--- Running Scenario: {scenario_name} ---")

        fixed_param = scenario_params["fixed_param"]
        varied_param = scenario_params["varied_param"]

        for varied_value in scenario_params["varied_range"]:
            # Set up dataset dimensions for this run
            if fixed_param == "n_samples":
                n_samples = scenario_params["fixed_value"]
                n_features = varied_value
            else:
                n_samples = varied_value
                n_features = scenario_params["fixed_value"]

            print(f"\nGenerating data: {n_samples} samples, {n_features} features")
            X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=20,
                                       n_redundant=50, random_state=42)

            for name, base_estimator in estimators.items():
                for i in range(N_REPEATS):
                    print(f"  Benchmarking {name} (Run {i+1}/{N_REPEATS})...")
                    try:
                        estimator = clone(base_estimator)
                        runtime, peak_mem = run_single_benchmark(estimator, X, y)
                        results.append({
                            "scenario": scenario_name, "algorithm": name,
                            "n_samples": n_samples, "n_features": n_features,
                            "runtime_sec": runtime, "peak_memory_mb": peak_mem
                        })
                    except Exception:
                        warnings.warn(f"  > FAILED: {name} on {n_samples}x{n_features}.",
                                      "Reason: {type(e).__name__}: {e}", UserWarning)

    # --- Save results to CSV ---
    df = pd.DataFrame(results)
    output_file = "benchmark_results_with_memory.csv"
    df.to_csv(output_file, index=False)
    print(f"\nBenchmarking complete. Results saved to '{output_file}'")

if __name__ == "__main__":
    main()

In [2]:
!git commit -a -m "updated TuRF slightly"
!git push

[main 099564b] updated TuRF slightly
Enumerating objects: 32, done.
Counting objects: 100% (24/24), done.
Delta compression using up to 20 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (14/14), 3.20 KiB | 819.00 KiB/s, done.
Total 14 (delta 10), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (10/10), completed with 7 local objects.
To https://github.com/GavinLynch04/FastSelect.git
   6d6f6c5..099564b  main -> main


In [23]:
!rm -rf dist/ build/ fast_select.egg-info/

In [24]:
!python3 -m build

* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools>=61.0
* Getting build dependencies for sdist...
running egg_info
writing src/fast_select.egg-info/PKG-INFO
writing dependency_links to src/fast_select.egg-info/dependency_links.txt
writing requirements to src/fast_select.egg-info/requires.txt
writing top-level names to src/fast_select.egg-info/top_level.txt
reading manifest file 'src/fast_select.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/fast_select.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing src/fast_select.egg-info/PKG-INFO
writing dependency_links to src/fast_select.egg-info/dependency_links.txt
writing requirements to src/fast_select.egg-info/requires.txt
writing top-level names to src/fast_select.egg-info/top_level.txt
reading manifest file 'src/fast_select.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/fast_select.egg-inf

In [25]:
!TWINE_USERNAME=__token__ TWINE_PASSWORD= python3 -m twine upload --repository testpypi dist/* --verbose


Uploading distributions to https://test.pypi.org/legacy/
INFO     dist/fast_select-0.1.3-py3-none-any.whl (20.1 KB)                      
INFO     dist/fast_select-0.1.3.tar.gz (22.1 KB)                                
INFO     username set by command options                                        
INFO     password set by command options                                        
INFO     username: __token__                                                    
INFO     password: <hidden>                                                     
Uploading fast_select-0.1.3-py3-none-any.whl
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 kB • 00:00 • 166.4 MB/s
INFO     Response from https://test.pypi.org/legacy/:                           
         200 OK                                                                 
INFO     <html>                                                                 
          <head>                                                                
        

In [ ]:
# plot_benchmarks.py
import sys

import matplotlib.pyplot as plt
import seaborn as sns


def plot_scenario(df, scenario_name, x_axis, y_axis, y_label,
                  title, filename, use_log_scale=True):
    """
    Generic helper function to generate and save a plot for a given scenario.

    Args:
        df (pd.DataFrame): The full results dataframe.
        scenario_name (str): The name of the scenario to filter for (e.g., 'p >> n').
        x_axis (str): The column name for the x-axis (e.g., 'n_features').
        y_axis (str): The column name for the y-axis (e.g., 'runtime_sec').
        y_label (str): The descriptive label for the y-axis.
        title (str): The main title for the plot.
        filename (str): The output filename for the saved plot.
        use_log_scale (bool): Whether to use a logarithmic scale for the y-axis.
    """
    # Filter the DataFrame for the specific scenario
    scenario_df = df[df['scenario'] == scenario_name].copy()

    # Create a new figure and axes for the plot
    plt.figure(figsize=(12, 8))

    # Use seaborn for a clean, publication-quality line plot
    sns.lineplot(
        data=scenario_df,
        x=x_axis,
        y=y_axis,
        hue='algorithm',
        marker='o',
        linewidth=2.5,
        errorbar='sd' # 'ci' is deprecated; 'errorbar' is the new standard
    )

    # Set plot properties
    plt.title(title, fontsize=18, fontweight='bold', pad=20)
    plt.xlabel(x_axis.replace('_', ' ').title(), fontsize=14)
    plt.ylabel(y_label, fontsize=14)

    if use_log_scale:
        plt.yscale('log')

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True, which="both", ls="--", c='0.7')
    plt.legend(title='Algorithm', fontsize=11, title_fontsize=13)
    plt.tight_layout()

    # Save the figure
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Plot saved to '{filename}'")
    plt.close() # Close the figure to free up memory

def main():
    """Main function to load results and generate all plots."""
    input_file = "benchmark_results_with_memory.csv"
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"Error: '{input_file}' not found.", file=sys.stderr)
        print("Please run the updated benchmark script first.", file=sys.stderr)
        sys.exit(1)

    # Validate that the necessary columns exist
    required_cols = ['scenario', 'algorithm', 'n_samples', 'n_features',
                     'runtime_sec', 'peak_memory_mb']
    if not all(col in df.columns for col in required_cols):
        print(f"Error: The CSV file '{input_file}' is missing required columns.",
              file=sys.stderr)
        print(f"Expected columns: {required_cols}", file=sys.stderr)
        print(f"Found columns: {list(df.columns)}", file=sys.stderr)
        sys.exit(1)

    # Use a professional plot style
    sns.set_theme(style="whitegrid")

    # --- Runtime Plots ---
    print("\n--- Generating Runtime Plots ---")
    plot_scenario(
        df=df,
        scenario_name='p >> n (Features Dominant)',
        x_axis='n_features',
        y_axis='runtime_sec',
        y_label="Runtime (seconds, log scale)",
        title='Benchmark: Runtime vs. Number of Features (p >> n)\n(n_samples fixed)',
        filename='benchmark_p_dominant_runtime.png',
        use_log_scale=True
    )
    plot_scenario(
        df=df,
        scenario_name='n >> p (Samples Dominant)',
        x_axis='n_samples',
        y_axis='runtime_sec',
        y_label="Runtime (seconds, log scale)",
        title='Benchmark: Runtime vs. Number of Samples (n >> p)\n(n_features fixed)',
        filename='benchmark_n_dominant_runtime.png',
        use_log_scale=True
    )

    # --- Memory Usage Plots ---
    print("\n--- Generating Memory Usage Plots ---")
    plot_scenario(
        df=df,
        scenario_name='p >> n (Features Dominant)',
        x_axis='n_features',
        y_axis='peak_memory_mb',
        y_label="Peak Memory Usage (MB, log scale)",
        title='Benchmark: Memory vs. Number of Features (p >> n)\n(n_samples fixed)',
        filename='benchmark_p_dominant_memory.png',
        use_log_scale=True # Memory can also vary greatly, log scale is often useful
    )
    plot_scenario(
        df=df,
        scenario_name='n >> p (Samples Dominant)',
        x_axis='n_samples',
        y_axis='peak_memory_mb',
        y_label="Peak Memory Usage (MB, log scale)",
        title='Benchmark: Memory vs. Number of Samples (n >> p)\n(n_features fixed)',
        filename='benchmark_n_dominant_memory.png',
        use_log_scale=True
    )

if __name__ == "__main__":
    main()